### 적절한 형태소 분석기 찾기

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Hannanum,Okt,Kkma
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from keybert import KeyBERT
from queue import Queue
from threading import Thread
docs = pd.read_parquet('./data/bookInfo2.parquet')
stopwords = pd.read_csv('./data/stopwords.csv')

In [2]:
def removeStopwords(text: list, stopwords: list) -> str:
    # text = list(filter(None, doc.split(" ")))
    word = [word for word in text if word not in stopwords]
    result = " ".join(word)
    return result

In [103]:
docs[docs['col1'].isin([9791160508796])] ### 점프 투 파이썬
idx = 631  ### 점프 투 파이썬

In [104]:
doc = ' '.join(docs.iloc[idx].astype(str).tolist()) 
# doc = ' '.join(docs.iloc[[idx]]['col4'].astype(str).tolist())
doc: str = (
    re.sub("\d[.]|\d|\W|[_]", " ", doc)
    .replace("머신 러닝", "머신러닝")
    .replace("인공 지능", "인공지능")
)
doc = list(filter(None, doc.split(" ")))
doc: str = removeStopwords(doc, stopwords)

In [105]:
# 우수한 형태소 분석기 추출 => 결과 hanNouns


#-- 기본 개념 --#
# morphs 형태소 단위
# nouns 단어
# pos 텍스트에 품사정보 포함

#-- Hannanum --#

hannanum = Hannanum()
hanMorphs = hannanum.morphs(doc)
hanNouns = hannanum.nouns(doc)
hanPos = hannanum.pos(doc)

#-- kkma --#
kkma = Kkma()
kkmaMorphs = kkma.morphs(doc)
kkmaNouns = kkma.nouns(doc)
kkmaPos = kkma.pos(doc)

#-- Okt --#
okt = Okt()
oktMorphs = okt.morphs(doc)
oktNouns = okt.nouns(doc)
oktPos = okt.pos(doc)


In [ ]:
item1 = pd.DataFrame(hanNouns).value_counts()
a = item1.index.tolist()
item1[np.array(list(map(lambda x: len(x[0]),a)))>1][:20]

item2= pd.DataFrame(oktNouns).value_counts()
a = item2.index.tolist()
item2[np.array(list(map(lambda x: len(x[0]),a)))>1][:20]

In [6]:
ha = pd.DataFrame(hanNouns).value_counts()
km = pd.DataFrame(kkmaNouns).value_counts()
ok = pd.DataFrame(oktNouns).value_counts()
konlpywords = pd.concat([ha,km,ok],axis=1)
konlpywords.columns = ['han', 'kkma','okt']


In [7]:
keyBertModel = KeyBERT("paraphrase-multilingual-MiniLM-L12-v2")


In [110]:
finish = []
for num, testType in enumerate([hanMorphs,hanNouns,kkmaMorphs,kkmaNouns,oktMorphs,oktNouns]) :
    words = ' '.join(testType)
    docResult: list = keyBertModel.extract_keywords(
        words, top_n=10, keyphrase_ngram_range=(3, 3), use_mmr=True, diversity=0.5
    )
    result = list(map(lambda x: x[0], docResult))
    finish.append(result)

keyBertResult = pd.DataFrame(finish).T
keyBertResult.columns = 'hanMorphs','hanNouns','kkmaMorphs','kkmaNouns','oktMorphs','oktNouns'

In [111]:
keyBertResult

,hanMorphs,hanNouns,kkmaMorphs,kkmaNouns,oktMorphs,oktNouns
0,기술 gt 개정판,개발자 리액트 핵심,액트 버전 hooks,도구 버전 업그레이드,또한 리액트 컴포넌트,개발자 리액트 핵심
1,usepreloader hook 만들,다양 미들웨어 서버,이자 현업 프로그래머,액트 기술 리액트를,도구 버전 업그레이드,사용 파일 수정
2,contexttype 사용 정리,라이브러리 사용 파일,파일 수정 하기,데이터베이스 바꾸기 베이스,todoscontainer hooks 전환,방식 매우 변화
3,적용 redux devtools,데이터 삭제 수정,도구 버전 업그레이드,추천 추천평 이론,렌더링 작업 usepreloader,컴포넌트 사용 이유
4,사용 리액트 라이브러리,사용 사용 수정하기,웨어 사용 redux,현업 프로그래머 활용,또는 static contexttype,또한 미들웨어 서버
5,리덕스 사용 리액트,버전 기능 도입,mongoose 이용 mongodb,개정판 버전에 기능,editorcontainer 만들기 tagboxcontainer,가짜 데이터 포스트
6,도구 버전 업그레이드,함수 함수형 업데이트,기본 설정 defaultprops,충돌 엔트리 웹팩,ref 설정 적용,리덕스 사용 리액트
7,함수형 컴포넌트 에서도,환경 설정 에디터,amp amp 사용,사용자 정보 권한,모니터링 느려지는 원인,개정판 리액트 버전
8,버전 hooks 라는,기술 개정판 리액트,리액트 활용 다양,환경 설정 에디터,amp amp 사용,작성 개선 배포
9,디자인 mongodb 서버,컬렉션 로그인 사용,history withrouter switch,개선 초판 수록,디자인 mongodb 서버,사용 리액트 라이브러리


### 웹 크롤러 예전버전

In [ ]:
def kyoboExtract(ISBN:str) -> dict : 
    kyoboUrl = f'http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}'
    kyoboHtml = requests.get(kyoboUrl)
    kyoboSoup = BeautifulSoup(kyoboHtml.content)

    #-- title --#
    bookTitle : str = kyoboSoup.h1.strong.string.strip()

    #-- book Intro --#
    bookIntro : str = kyoboSoup.find('div',{'class':'box_detail_article'}).string.strip()

    #-- author Intro --#
    authorIntro : str = kyoboSoup.find('div',{'class' : 'box_detail_article'}).string.strip()

    #-- keywords --#
    keywords = kyoboSoup.find('div',{'class':'tag_list'}).find_all('a')
    keywords : list = [k.string.replace('#','') for k in keywords]

    #-- TOC --#
    toc = kyoboSoup.find_all('div',{'class' : 'box_detail_article'})[3]
    toc = re.sub('<.*?>|\\t','',str(toc))
    toc = re.sub('\\n',' ',toc)
    toc :str = re.sub('\\r',',',toc)
    toc = re.sub('\d[.]|\d|[,]\s*','',toc) 
    
    return dict(bookTitle=bookTitle,bookIntro=bookIntro,authorIntro=authorIntro,toc=toc,keywords=keywords)


kyoboExtract(9791160505979)


### lib info extraction

In [2]:
import pandas as pd
a = pd.read_excel('./data/lib_info.xlsx',)
a = a[6:]
a.columns = a.iloc[0]
a = a[1:]
a = a[['도서관명','도서관코드']].reset_index(drop=True)
a.columns= ['libName','libCode']
a.to_csv('./data/lib_info.csv',encoding='CP949')

/Users/yangwoolee/Library/Python/3.8/lib/python/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
import os 

dataList=os.listdir(path='./data/libbooks/')

# 결과 종합하기
result = []
for num,dataName in enumerate(dataList) :

    # 불러오기
    itemCol = ['도서명','저자','출판사','발행년도','ISBN','주제분류번호','도서권수','대출건수','등록일자']
    LibBook = pd.read_csv(f'./data/libbooks/{dataName}',encoding='CP949',usecols=itemCol)

    #필터링
    BM = LibBook['등록일자'] > '2018-01-01'
    LibBook = LibBook[BM]

    # 004,005 분류하기
    k = LibBook['주제분류번호'].astype(str)
    item= k[k.str.contains('004.')].index.tolist()
    itemAppend=k[k.str.contains('005.')].index.tolist()
    item.extend(itemAppend)

    #booklist 만들기 
    bookList=LibBook[LibBook.index.isin(item)]
    bookList['도서관ID'] =num
    result.append(bookList)

# Concat & Save BookListRaw
final = pd.concat(result).reset_index(drop=True)
final.to_csv('./data/bookListRaw.csv',encoding='CP949')

# Save BookList
b = final.drop_duplicates(subset='ISBN')
dropDuplicate = b[['도서명','저자','출판사','ISBN','주제분류번호','등록일자']].reset_index(drop=True)

dropDuplicate.to_csv('./data/bookListSearch',encoding='CP949')



In [ ]:
# 구버전 문장 검색기
# import ast
# kkk = pd.read_csv('./data/keywordExtraction2.csv',encoding='cp949',index_col=0)
# lists = kkk['keywords'].tolist()

# def changeStringToList(strList) :
#     # strList = strList.replace(' ',', ')
#     # result = np.array(ast.literal_eval(strList))
#     return ast.literal_eval(strList)

# # 중복값 찾기
# def findOverlapNum(keywordsOfBook:list,keywordsWord2Vec):
#     return np.in1d(keywordsWord2Vec,keywordsOfBook)


# def searchByKeywords(words:list) :
#     #검색 단어 갯수
#     wordsLen = len(words)

#     #model load(나중에 밖으로 뺴내야할 듯)
#     loaded_model = KeyedVectors.load_word2vec_format("booksTest1")

#     #키워드 단어 불러오기 20개 추출
#     keywordsWord2Vec = loaded_model.most_similar(positive=words,topn=20)
#     Word2VecKeyword = list(map(lambda x : x[0],keywordsWord2Vec))

#     # 사용자가 검색한 단어와 합치기
#     words.extend(Word2VecKeyword)

#     # 키워드 일치여부 확인하고 사용자 검색 단어에 가중치 넣기
#     val = np.array(list(map(lambda x : findOverlapNum(x,words) ,result)))
#     df = pd.DataFrame(val)
#     for i in range(wordsLen) :
#         df[i] = df[i]*3
        
#     return df.T.sum()


#  #-- 결과 --#   
# # df['keywords']의 String을 List로 반환
# result = list(map(changeStringToList,lists))

# kkkk=kkk.copy()
# userkeywords = ['데이터베이스','오라클'] 
# kkkk['sums'] = searchByKeywords(userkeywords)
# kkkk[kkkk['sums'] >= 3][['도서명','ISBN','sums']].sort_values(by='sums',ascending=False).head(15)